### Landing Surface - TLOF (Touchdown and Lift-Off Area)  Parameters

In [ ]:
import os
import time
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Prompt user to enter a natural language instruction
user_input = input("Describe the FATO in natural language (e.g., 'Create a FATO JSON for...'):\n")

# Function definition to extract structured fields from input
function_def = [
    {
        "name": "generate_fato_layout",
        #"description": "Generate detailed FATO layout from helicopter and location data, Sometimes FATO are also called as Geometry",
        "description": "Generate a structured FATO (Final Approach and Takeoff) layout in JSON format based on aircraft and location data. If required values (e.g., aircraft type, position, or shape type) are not provided, request them from the user. The layout includes geometric and visual details such as size, shape, markings, rotation, and transparency. Use realistic defaults if data is missing (e.g., elevation between 10–30 meters). This layout is intended for use in a larger helipad planning or rendering system and must follow aviation-appropriate standards. Output must match the expected JSON structure with no null or zero values unless valid.",
        "parameters": {
            "type": "object",
            "properties": {
                "aircraft": {"type": "string"},
                "fato_length": {"type": "number"},
                "fato_width": {"type": "number"},
                "fato_height": {"type": "number"},
                "sides": {"type": "number"},
                "diameter": {"type": "number"},
                "marking_thickness": {"type": "number"},
                "rotation": {"type": "number"},
                "isVisible": { "type": "boolean" },
                "transparency": {"type": "number"},
                "elevation": {"type": "number"},
                "shape_type": {
                    "type": "string",
                    "enum": ["Circle", "Polygon", "Rectangle"]
                },
                "position": {
                    "type": "array",
                    "items": {"type": "number"},
                    "minItems": 2,
                    "maxItems": 2
                },



                "layerName": { "type": "string" },
                "netHeight": { "type": "number" },
                "curveAngle": { "type": "number" },
                "dashLength": { "type": "number" },
                "lightColor": { "type": "string" },
                "lightScale": { "type": "number" },
                "multiplier": { "type": "number" },
                "lightHeight": { "type": "number" },
                "lightRadius": { "type": "number" },
                "markingType": { "type": "string" },
                "dashDistance": { "type": "number" },
                "markingColor": { "type": "string" },
                "lightCategory": { "type": "boolean" },
                "lightDistance": { "type": "number" },
                "scaleCategory": { "type": "boolean" },
                "textureScaleU": { "type": "number" },
                "textureScaleV": { "type": "number" },
                "vertipadColor": { "type": "string" },
                "offsetDistance": { "type": "number" },
                "safetyAreaType": { "type": "string" },
                "safetyNetColor": { "type": "string" },
                "safetyNetScaleU": { "type": "number" },
                "safetyNetScaleV": { "type": "number" },
                "aircraftCategory": { "type": "boolean" },
                "markingsCategory": { "type": "boolean" },
                "safetyNetCategory": { "type": "boolean" },
                "safetyAreaCategory": { "type": "boolean" },
                "safetyNetTransparency": { "type": "number" }

            },
            "required": [
                "aircraft", "fato_length", "fato_width","fato_height", "sides",
                "diameter", "marking_thickness", "rotation", "transparency",
                #"elevation", 
                "shape_type", "position"
            ]
        }
    }
]


# Call OpenAI chat completion
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    messages=[
        #{"role": "system", "content": "If elevation is not provided, assume 20 meters as default."},
        {"role": "user", "content": user_input}
    ],
    functions=function_def,
    function_call={"name": "generate_fato_layout"}
)

# Extract function arguments from the response
function_args = json.loads(response.choices[0].message.function_call.arguments)

# Set default elevation if not provided or invalid (None or 0)
if "elevation" not in function_args or function_args["elevation"] in (None, 0):
    function_args["elevation"] = 20.0



# Set lightCategory = True if any light property is explicitly changed from default
default_light_values = {
    "lightColor": "white",
    "lightScale": 1,
    "lightDistance": 1,
    "lightRadius": 0.3,
    "lightHeight": 0.2
    
}

light_category_enabled = any(
    function_args.get(k) is not None and function_args.get(k) != v
    for k, v in default_light_values.items()
)
allowed_light_colors = {
    "white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"
}

# Get and normalize color
input_color = function_args.get("lightColor", "white").lower()

# Check and warn
if input_color not in allowed_light_colors:
    print(
        f"⚠️Unsupported lightColor '{input_color}'. Allowed: {', '.join(sorted(allowed_light_colors))}. Using default: 'white'."
    )
    input_color = "white"

#print(function_args)

# Create unique ID
fato_id = f"FATO-{int(time.time() * 1000)}"

# Build final JSON object
fato_json = {
    "id": fato_id,
    "groupId": None,
    "position": function_args["position"],  # [lon, lat]
    "isVisible": True,
    "layerName": "GEOMETRY_FATO",
    "layerOrder": 0,
    "dimensions": {
        "sides": max(3, min(function_args["sides"], 70)), #Shape Sides
        "width": max(1, min(function_args["fato_width"], 300)),
        "dValue": 10,
                "height": max(1, min(function_args["fato_height"], 300)), #Thickness        
                "length": max(1, min(function_args["fato_length"], 300)),
        "aircraft": function_args["aircraft"],
                "diameter": max(1.0, min(function_args["diameter"], 200.0)), #Diameter
                "rotation": max(0, min(function_args["rotation"], 359)), #Rotation
        "isVisible": True,
        "layerName": "GEOMETRY_FATO",
        "netHeight": 15,
        "shapeType": function_args["shape_type"],
                "baseHeight": max(0.0, min(function_args["elevation"], 1000.0)), #Elevation
        "curveAngle": 45,
        "dashLength": 1,
        "multiplier": 1.5,
        "markingType": "dashed",
        "dashDistance": 1,
        "markingColor": "white",
                "transparency": max(0, min(function_args["transparency"], 1)), #Transparency
        "lightCategory": light_category_enabled, #Lighting setting on/off
        "lightColor": input_color.lower(), #Light Color
        "lightScale": max(-20, min(function_args.get("lightScale") or 1, 100)), #Perimeter Edge Offset
        "lightDistance": max(0.5, min(function_args.get("lightDistance") or 1, 50)), #Distance between Lights
        "lightRadius": max(0.1, min(function_args.get("lightRadius") or 0.3, 1)), #Lights Base Radius
        "lightHeight": max(0.1, min(function_args.get("lightHeight") or 0.2, 0.25)), #Light Extrusion Height
        "scaleCategory": False,
        "textureScaleU": 1,
        "textureScaleV": 1,
        "vertipadColor": "#808080",
        "offsetDistance": 3,
        "safetyAreaType": "multiplier",
        "safetyNetColor": "#FF0000",
        "safetyNetScaleU": 1,
        "safetyNetScaleV": 1,
        "aircraftCategory": False,
        "markingThickness": max(0.01, min(function_args["marking_thickness"], 5.0)),
        "markingsCategory": False,
        "safetyNetCategory": False,
        "safetyAreaCategory": False,
        "safetyNetTransparency": 0.5
    }
}

# Output only the JSON
print(json.dumps(fato_json, indent=2))

Describe the FATO in natural language (e.g., 'Create a FATO JSON for...'):
  Build a visible FATO with width 30, height 10, and rotated 30 degrees.


{
  "id": "FATO-1750747643465",
  "groupId": null,
  "position": [
    0,
    0,
    0
  ],
  "isVisible": true,
  "layerName": "GEOMETRY_FATO",
  "layerOrder": 0,
  "dimensions": {
    "sides": 4,
    "width": 30,
    "dValue": 10,
    "height": 10,
    "length": 30,
    "aircraft": "Generic",
    "diameter": 1.0,
    "rotation": 30,
    "isVisible": true,
    "layerName": "GEOMETRY_FATO",
    "netHeight": 15,
    "shapeType": "Rectangle",
    "baseHeight": 20.0,
    "curveAngle": 45,
    "dashLength": 1,
    "multiplier": 1.5,
    "markingType": "dashed",
    "dashDistance": 1,
    "markingColor": "white",
    "transparency": 0,
    "lightCategory": false,
    "lightColor": "white",
    "lightScale": 1,
    "lightDistance": 1,
    "lightRadius": 0.3,
    "lightHeight": 0.2,
    "scaleCategory": false,
    "textureScaleU": 1,
    "textureScaleV": 1,
    "vertipadColor": "#808080",
    "offsetDistance": 3,
    "safetyAreaType": "multiplier",
    "safetyNetColor": "#FF0000",
    "safety

In [ ]:
import os
import time
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Prompt user input
user_input = input("Describe the TLOF (Landing Surface) in natural language:\n")

# Function definition
function_def = [
    {
        "name": "generate_landing_surface_layout",
        "description": "Generate a structured TLOF (Touchdown and Lift-Off Area) JSON layout with visual and dimensional properties. Defaults will be applied where values are missing.",
        "parameters": {
            "type": "object",
            "properties": {
                "aircraft": {"type": "string"},
                "surface_length": {"type": "number"},
                "surface_width": {"type": "number"},
                "surface_height": {"type": "number"},
                "sides": {"type": "number"},
                "diameter": {"type": "number"},
                "marking_thickness": {"type": "number"},
                "rotation": {"type": "number"},
                "transparency": {"type": "number"},
                "elevation": {"type": "number"},
                "shape_type": {
                    "type": "string",
                    "enum": ["Circle", "Polygon", "Rectangle"]
                },
                "position": {
                    "type": "array",
                    "items": {"type": "number"},
                    "minItems": 2,
                    "maxItems": 2
                },
                "lightColor": {"type": "string"},
                "lightScale": {"type": "number"},
                "lightDistance": {"type": "number"},
                "lightRadius": {"type": "number"},
                "lightHeight": {"type": "number"},
                "markingType": {"type": "string"},
                "dashDistance": {"type": "number"},
                "markingColor": {"type": "string"}
            },
            "required": [
                "aircraft", "surface_length", "surface_width", "surface_height",
                "sides", "diameter", "marking_thickness", "rotation", "transparency",
                "shape_type", "position"
            ]
        }
    }
]

# Get AI response
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    messages=[{"role": "user", "content": user_input}],
    functions=function_def,
    function_call={"name": "generate_landing_surface_layout"}
)

# Parse function arguments
function_args = json.loads(response.choices[0].message.function_call.arguments)

# Defaults
function_args["elevation"] = function_args.get("elevation") or 20.0
allowed_light_colors = {"white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"}
input_color = function_args.get("lightColor", "white").lower()
if input_color not in allowed_light_colors:
    print(f"⚠️ Unsupported lightColor '{input_color}', defaulting to 'white'.")
    input_color = "white"

# Light category logic
default_light_values = {
    "lightColor": "white",
    "lightScale": 1,
    "lightDistance": 1,
    "lightRadius": 0.3,
    "lightHeight": 0.2
}
light_category_enabled = any(
    function_args.get(k) is not None and function_args.get(k) != v
    for k, v in default_light_values.items()
)

# ID
tlof_id = f"TLOF-{int(time.time() * 1000)}"

# Final JSON
tlof_json = {
    "TLOF": [
        {
            "position": function_args["position"],
            "dimensions": {
                "unit": "m",
                "sides": max(3, min(function_args["sides"], 70)),
                "width": max(1, min(function_args["surface_width"], 300)),
                "dValue": 10,
                "height": max(0.01, min(function_args["surface_height"], 300)),
                "length": max(1, min(function_args["surface_length"], 300)),
                "aircraft": function_args["aircraft"],
                "diameter": max(1.0, min(function_args["diameter"], 200.0)),
                "rotation": max(0, min(function_args["rotation"], 359)),
                "tdpcType": "circle",
                "isVisible": True,
                "layerName": "Praveen_TLOF",
                "netHeight": 15,
                "shapeType": function_args["shape_type"],
                "tdpcColor": "orange",
                "tdpcScale": 5,
                "baseHeight": max(0.0, min(function_args["elevation"], 1000.0)),
                "curveAngle": 45,
                "dashLength": 1,
                "lightColor": input_color,
                "lightScale": max(-20, min(function_args.get("lightScale") or 1, 100)),
                "multiplier": 1.5,
                "lightHeight": max(0.1, min(function_args.get("lightHeight") or 0.2, 0.25)),
                "lightRadius": max(0.1, min(function_args.get("lightRadius") or 0.3, 1)),
                "markerColor": "green",
                "markerScale": 1,
                "markingType": function_args.get("markingType", "dashed"),
                "dashDistance": function_args.get("dashDistance", 1),
                "markingColor": function_args.get("markingColor", "white"),
                "tdpcCategory": True,
                "tdpcRotation": 0,
                "transparency": max(0, min(function_args["transparency"], 1)),
                "landingMarker": "H",
                "lightCategory": light_category_enabled,
                "lightDistance": max(0.5, min(function_args.get("lightDistance") or 1, 50)),
                "scaleCategory": False,
                "tdpcExtrusion": 0.02,
                "tdpcThickness": 0.5,
                "textureScaleU": 1,
                "textureScaleV": 1,
                "vertipadColor": "#808080",
                "markerRotation": 0,
                "offsetDistance": 3,
                "safetyAreaType": "multiplier",
                "safetyNetColor": "#FF0000",
                "letterThickness": 0.5,
                "markerThickness": 0.02,
                "safetyNetScaleU": 1,
                "safetyNetScaleV": 1,
                "aircraftCategory": False,
                "markingThickness": max(0.01, min(function_args["marking_thickness"], 5.0)),
                "markingsCategory": False,
                "safetyNetCategory": False,
                "safetyAreaCategory": False,
                "landingMarkerCategory": True,
                "safetyNetTransparency": 0.5
            }
        }
    ]
}

# Print output
print(json.dumps(tlof_json, indent=2))
